<a href="https://colab.research.google.com/github/hani-haddad/testpy/blob/master/profile_scraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import json
import os.path
import time
import regex as re
import pandas as pd

In [15]:
page=requests.get("https://keithgalli.github.io/web-scraping/webpage.html")
page

<Response [200]>

In [ ]:
soup = BeautifulSoup(page.content,'html.parser')
print(soup.prettify())

In [ ]:
some = soup.find_all(name=re.compile(".*"),string= re.compile("[Ss]ome"))
some[3].text

In [ ]:
soup.select("h1 + p")    # check css selectors https://www.w3schools.com/cssref/css_selectors.php

[<p>Link to more interesting example: <a href="https://keithgalli.github.io/web-scraping/webpage.html">keithgalli.github.io/web-scraping/webpage.html</a></p>]

In [ ]:
soup.select("p#paragraph-id > b") #select by id 

[<b>Some bold text</b>]

In [ ]:
s=soup.select("body > p")

for i in s:
  print(i.select("i"))

[<i>Some italicized text</i>]
[]


In [ ]:
#if multiple  child elements use get_text to return all the text in each child 

In [27]:
#get all text in page using find_next_siblings()/find_previous_siblings()

soup.find("body").text.strip().replace("/n","")
header = soup.body.find("h2", string="Photos")
previous_elements = header.find_previous_siblings()
previous_elements_sorted = previous_elements[::-1]
elements = [x.get_text() for x in previous_elements_sorted]
text = "\n".join(elements)
print(text)

Welcome to my page!

About me
Hi, my name is Keith and I am a YouTuber who focuses on content related to programming, data science, and machine learning!
Here is a link to my channel: youtube.com/kgmit
I grew up in the great state of New Hampshire here in the USA. From an early age I always loved math. Around my senior year of high school, my brother first introduced me to programming. I found it a creative way to apply the same type of logical thinking skills that I enjoyed with math. This influenced me to study computer science in college and ultimately create a YouTube channel to share some things that I have learned along the way.
Hobbies
Believe it or not, I don't code 24/7. I love doing all sorts of active things. I like to play ice hockey & table tennis as well as run, hike, skateboard, and snowboard. In addition to sports, I am a board game enthusiast. The two that I've been playing the most recently are Settlers of Catan and Othello.
Fun Facts

Owned my dream car in high schoo

##Grabing all of the social links from the webpage
**in** 3 different ways

In [ ]:
socials= soup.find_all(class_= "socials")[0].select("li")

for i in socials:
  print(i.get_text())

Instagram: https://www.instagram.com/keithgalli/
Twitter: https://twitter.com/keithgalli
LinkedIn: https://www.linkedin.com/in/keithgalli/
TikTok: https://www.tiktok.com/@keithgalli


In [ ]:
for ul in soup.find_all('ul', { 'class': 'socials' }):
	values = [li.text for li in ul.find_all('li')]
  
values

['Instagram: https://www.instagram.com/keithgalli/',
 'Twitter: https://twitter.com/keithgalli',
 'LinkedIn: https://www.linkedin.com/in/keithgalli/',
 'TikTok: https://www.tiktok.com/@keithgalli']

In [ ]:
links=soup.select("li.social a")
l= [link["href"] for link in links ]
l

['https://www.instagram.com/keithgalli/',
 'https://twitter.com/keithgalli',
 'https://www.linkedin.com/in/keithgalli/',
 'https://www.tiktok.com/@keithgalli']

##table scraping 

In [ ]:
table=soup.find("table", {'class': 'hockey-stats'})
table

In [ ]:
from pandas.io.common import dataclasses

columnsN = [th.text for th in table.thead.tr.find_all('th')]
data=[]
rows=table.find("tbody").find_all('tr')
for row in rows:
 t=[td.text.strip() for td in row.find_all('td') ]
 data.append(t)

tableDF=pd.DataFrame(data,columns=columnsN)
tableDF

,S,Team,League,GP,G,A,TP,PIM,+/-,,POST,GP,G,A,TP,PIM,+/-
0,2014-15,MIT (Mass. Inst. of Tech.),ACHA II,17,3,9,12,20,,|,,,,,,,
1,2015-16,MIT (Mass. Inst. of Tech.),ACHA II,9,1,1,2,2,,|,,,,,,,
2,2016-17,MIT (Mass. Inst. of Tech.),ACHA II,12,5,5,10,8,0,|,,,,,,,
3,2017-18,Did not play,,,,,,,,|,,,,,,,
4,2018-19,MIT (Mass. Inst. of Tech.),ACHA III,8,5,10,15,8,,|,,,,,,,


##Grab all fun facts that use word "is"
in 2 different ways

In [ ]:
for ul in soup.find_all('ul', { 'class': 'fun-facts' }):
  values=[li for li in ul.find_all(string=re.compile("is")) ]
values

['Middle name is Ronald',
 'Dunkin Donuts coffee is better than Starbucks',
 'A favorite book series of mine is ',
 'Current video game of choice is ',
 "The band that I've seen the most times live is the "]

In [ ]:
facts = soup.select("ul.fun-facts li")
facts_with_is = [fact.find(string=re.compile("is")) for fact in facts]
facts_with_is = [fact.find_parent().get_text() for fact in facts_with_is if fact]
facts_with_is

['Middle name is Ronald',
 'Dunkin Donuts coffee is better than Starbucks',
 "A favorite book series of mine is Ender's Game",
 'Current video game of choice is Rocket League',
 "The band that I've seen the most times live is the Zac Brown Band"]

#images scraping and downloding 

In [ ]:
url = "https://keithgalli.github.io/web-scraping/webpage.html"
purl="https://keithgalli.github.io/web-scraping/"
def downloadImgs(pageurl, imageurl ):
  page = requests.get(pageurl)
  soup = BeautifulSoup(page.content,'html.parser')
  for img in soup.find_all("img"):
    scrapedimgurl=img["src"]
    imgurl=imageurl+re.sub("[.]/", "", scrapedimgurl)
    img_data = requests.get(imgurl).content
    with open(str(img)+".jpg", 'w') as f:
     f.write(img_data)

downloadImgs(url,purl)

##nested pages scraping 

In [ ]:
page = requests.get("https://keithgalli.github.io/web-scraping/webpage.html")
soup = BeautifulSoup(page.content,'html.parser')
for link in soup.select("div.block a"):
  l="https://keithgalli.github.io/web-scraping/"+ link["href"]
  nestedPage= requests.get(l)
  soup = BeautifulSoup(nestedPage.content,'html.parser')
  word= soup.select("p#secret-word")

  print(word[0].text)
